<a href="https://colab.research.google.com/github/Gaurav7004/Textual-Analysis-ILH/blob/main/QPR_Trained_Model_20th_Sept_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install torch
!pip install pandas
!pip install scikit-learn
!pip install nltk

import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import pandas as pd
import numpy as np
import torch

from nltk.tokenize import sent_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import (
    BertTokenizerFast,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments,
    BertConfig,
)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# =====================================
# Step 1: Prepare the Dataset
# =====================================

# Your CARS dataset in JSON format
cars_dataset = [
  {
    "event": "USAID partners with India to build equitable and inclusive healthcare access and climate-resilient health services.",
    "cause": "Recognizing the need for a robust healthcare system serving India's diverse population and aligning with India's National Health Policy.",
    "action": "Fostering partnerships, leveraging technology and innovation, and focusing on equitable healthcare access.",
    "result": "Improved health outcomes, strengthened health systems, and equitable access to quality healthcare.",
    "significance": "Demonstrates scalable global development solutions and enhances global health security."
  },
  {
    "event": "Creation of the world's first health-focused blended financing facility by USAID and local partners.",
    "cause": "Need for innovative financing to support healthcare enterprises and scale healthcare innovations.",
    "action": "Establishing a financing facility offering grants and debt financing to social entrepreneurs.",
    "result": "Invested in over 90 innovations, benefiting more than 43 million people and leveraging over $270 million in investments.",
    "significance": "Accelerated health innovation scaling and established a replicable financing model globally."
  },
  {
    "event": "Development and nationwide scaling of an AI tool to scan media for disease emergence.",
    "cause": "Need for enhanced disease surveillance and early outbreak detection.",
    "action": "Developing an AI tool to scan media for signs of disease and integrating it into national platforms.",
    "result": "Scanned 65 million articles in 2023, helping prevent an estimated 230 outbreaks.",
    "significance": "Improved global health security and provided a model for other countries."
  },
  {
    "event": "Scaling of maternal and child health models in India and Eastern Africa.",
    "cause": "High rates of infant respiratory distress and maternal deaths due to postpartum hemorrhage.",
    "action": "Demonstrated use of CPAP machines for infants and introduced new drugs and devices to reduce maternal deaths.",
    "result": "Improved maternal and child health outcomes and scaled health models in multiple regions.",
    "significance": "Contributed to reducing mortality rates and provided scalable health solutions globally."
  },
  {
    "event": "Transition of USAID HIV programs to models of entrepreneurship and empowerment for the LGBTQI+ community.",
    "cause": "Need to empower the LGBTQI+ community beyond traditional healthcare models and reduce stigma.",
    "action": "Established comprehensive clinics, supported civil society organizations, and promoted LGBTQI+ social enterprises.",
    "result": "Empowered the community with safe healthcare access and opportunities for self-sufficiency.",
    "significance": "Advanced equality, inclusion, and influenced global approaches to LGBTQI+ healthcare."
  },
  {
    "event": "Partnership with the Government of India to develop climate-resilient healthcare services.",
    "cause": "Impact of climate change on health and the necessity for sustainable healthcare infrastructure.",
    "action": "Developed solar-powered facilities, smart water usage, waste management, and greener supply chains; created state action plans.",
    "result": "Enhanced sustainability and resilience of healthcare services across 13 states.",
    "significance": "Set a precedent for integrating climate resilience into healthcare, aiding global efforts against climate change."
  },
  {
    "event": "Development of online platforms and networks to support young people's health decision-making.",
    "cause": "Need to improve access to healthcare information and empower youth in health decisions.",
    "action": "Helped develop digital platforms providing health information and products to youth.",
    "result": "Enhanced access to health resources and improved decision-making among young people.",
    "significance": "Empowered youth, contributing to long-term positive health outcomes and demonstrating effective use of technology."
  },
  {
    "event": "Facilitation of health learning exchanges between India and other countries.",
    "cause": "Need to share best practices and collaborate on global health challenges.",
    "action": "Organized exchanges with governments of Bangladesh, Fiji, Indonesia, Nepal, Nigeria, the Philippines, and Sri Lanka.",
    "result": "Enhanced collaboration, adoption of successful health practices, and strengthened global health networks.",
    "significance": "Promoted international cooperation, contributing to improved health outcomes globally."
  },
  {
    "event": "Embedding inclusive development principles in USAID's work at every level.",
    "cause": "Importance of diversity, equity, and accessibility in development programs.",
    "action": "Worked with community-based organizations and marginalized groups; implemented inclusive monitoring and evaluation processes.",
    "result": "Increased demand for equitable healthcare services and improved inclusivity in interventions.",
    "significance": "Strengthened impact of initiatives and set standards for inclusive practices in international development."
  },
  {
    "event": "USAID's efforts to enhance global health security and strengthen local health systems.",
    "cause": "Threat of global health crises and the need for robust local health systems.",
    "action": "Strengthened disease surveillance, responded to emerging infections, mobilized resources, and promoted research.",
    "result": "Improved ability to manage health challenges and prevent potential outbreaks.",
    "significance": "Reduced risk of global health emergencies and contributed to safer, healthier communities worldwide."
  },
  {
    "event": "USAID's collaboration to support TB elimination and HIV targets in alignment with India's health goals.",
    "cause": "The prevalence of TB and HIV in India and the need to meet elimination targets.",
    "action": "Supporting programs focused on TB elimination and HIV prevention and treatment.",
    "result": "Progress toward meeting TB elimination and HIV targets in India.",
    "significance": "Contributed to improved public health and aligned with national and global health priorities."
  },
  {
    "event": "USAID's work to ensure local voices guide the formulation of equitable and effective responses.",
    "cause": "Need for community engagement to enhance the effectiveness of health programs.",
    "action": "Collaborated with local communities, marginalized populations, and tribal groups to drive demand for comprehensive healthcare.",
    "result": "Programs that are more responsive to local needs and increased utilization of health services.",
    "significance": "Enhanced community ownership and sustainability of health interventions."
  },
  {
    "event": "Implementation of monitoring and evaluation processes to assess the impact of inclusive development.",
    "cause": "Need to measure the effectiveness of incorporating inclusive development into healthcare interventions.",
    "action": "Developed and implemented monitoring and evaluation frameworks focusing on inclusivity.",
    "result": "Obtained data on the impact of inclusive practices, informing future program improvements.",
    "significance": "Provided evidence to support the effectiveness of inclusive development, influencing policy and program design."
  },
  {
    "event": "USAID's support for telemedicine and mental health initiatives.",
    "cause": "Challenges in accessing healthcare services, especially in remote areas, and increasing mental health concerns.",
    "action": "Promoted telemedicine solutions and mental health programs through partnerships and technology.",
    "result": "Improved access to healthcare and mental health services for underserved populations.",
    "significance": "Addressed critical gaps in healthcare delivery and contributed to overall well-being."
  },
  {
    "event": "Advancement of innovative healthcare models and high-impact solutions through partnerships.",
    "cause": "Complex health challenges in India requiring novel approaches.",
    "action": "Tested and piloted healthcare models and innovations with public and private sector entities.",
    "result": "Development of scalable solutions to health challenges in India.",
    "significance": "Enabled replication of successful models worldwide, enhancing global health outcomes."
  }
]


# Convert the dataset into a DataFrame for easier manipulation
data_rows = []
for entry in cars_dataset:
    # Split each component into sentences (if necessary)
    for component in ['event', 'cause', 'action', 'result', 'significance']:
        text = entry[component]
        sentences = sent_tokenize(text)
        for sentence in sentences:
            data_rows.append({'sentence': sentence, 'label': component.capitalize()})

df = pd.DataFrame(data_rows)

# Map labels to integers
label_map = {'Event': 0, 'Cause': 1, 'Action': 2, 'Result': 3, 'Significance': 4}

df['label'] = df['label'].map(label_map)

# Shuffle the DataFrame
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Split the data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['sentence'].tolist(),
    df['label'].tolist(),
    test_size=0.1,
    random_state=42
)

# =====================================
# Step 2: Tokenization
# =====================================

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(
    train_texts,
    truncation=True,
    padding=True,
    max_length=128
)

val_encodings = tokenizer(
    val_texts,
    truncation=True,
    padding=True,
    max_length=128
)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# =====================================
# Step 3: Create Dataset Class
# =====================================

class CARSDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # Return CPU tensors
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CARSDataset(train_encodings, train_labels)
val_dataset = CARSDataset(val_encodings, val_labels)

# =====================================
# Step 4: Initialize the Model
# =====================================

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(label_map),
    output_attentions=False,
    output_hidden_states=False
).to(device)


# =====================================
# Step 5: Training Arguments
# =====================================

training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    num_train_epochs=5,              # Total number of training epochs
    per_device_train_batch_size=8,   # Batch size per device during training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    warmup_steps=50,                 # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=10,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
)

# =====================================
# Step 6: Define Metrics
# =====================================

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='weighted', zero_division=0
    )
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# =====================================
# Step 7: Initialize Trainer
# =====================================

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# =====================================
# Step 8: Train the Model
# =====================================

trainer.train()

# Save the model
trainer.save_model('./trained_model')
tokenizer.save_pretrained('./trained_model')

# =====================================
# Step 9: Evaluate the Model
# =====================================

eval_results = trainer.evaluate()
print(f"\nEvaluation results:\n{eval_results}\n")

# =====================================
# Step 10: Inference on New Texts
# =====================================

def extract_cars_components(text, model, tokenizer, label_map):
    # Split the text into sentences
    sentences = sent_tokenize(text)

    # Tokenize sentences
    encodings = tokenizer(
        sentences,
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors='pt'
    ).to(device)  # Move tensors to the appropriate device

    # Make predictions
    with torch.no_grad():
        outputs = model(**encodings)
        logits = outputs.logits
        predicted_class_ids = torch.argmax(logits, dim=1)
        predicted_labels = predicted_class_ids.cpu().numpy()

    # Map IDs to labels
    id_to_label = {v: k for k, v in label_map.items()}
    predicted_labels = [id_to_label[id] for id in predicted_labels]

    # Organize sentences by labels
    cars_components = {'Cause': [], 'Action': [], 'Result': [], 'Significance': [], 'Event': []}
    for sentence, label in zip(sentences, predicted_labels):
        if label in cars_components:
            cars_components[label].append(sentence)

    return cars_components


# Example usage:

# Load the trained model and tokenizer (if not already in memory)
# model = BertForSequenceClassification.from_pretrained('./trained_model').to(device)
# tokenizer = BertTokenizerFast.from_pretrained('./trained_model')

# Sample test text
test_text = """
Guidance: This section should briefly state the results the activity seeks to achieve and the relationship
of these results to the USAID/India’s goal of Ending Preventable Maternal and Child Death (EPCMD),
FP2030, TB-free India and AIDS-free generation in India. Provide clear and precise descriptions of the
guiding principle for this QPR, e.g. highlights of activities, key achievements of the reporting period,
major plans of the next quarter, etc. Restrict to TWO pages only.
USAID India with the MOMENTUM Country and Global Leadership (MCGL) field support mechanism is
implementing a program focused on Family Planning (FP) and Reproductive Health (RH) for young
populations (YPs) that will contribute to the achievement of national FP goals and is aligned with
USAID India’s Country Development Cooperation Strategies (CDCS) goal of improving the health and
resilience of India’s marginalized populations. The project’s vision is to demonstrate innovative
models to improve young people’s FP/RH outcomes. The project promotes the adaptation of
promising models of high-impact interventions demonstrated in previous programs. This project is
expected to catalyze lessons learned from USAID’s previous investments in FP/RH for youth. The
project will leave behind replicable and scalable models for young people so that they take care of
their own sexual reproductive health needs and well-being, irrespective of their socio-demographic
and economic differences. This program is aligned with USAID’s priority of leading India to achieve
progress towards sustainable development by building local institutional capacities, and steering
USAID India’s strategic approaches of promoting inclusive development, harnessing private sector
engagement, and leveraging high-level government of India commitments and leadership.
MCGL India-Yash interventions:
• Youth-led Movement: The intervention aims to build a multi-state youth-led movement for
Family Planning and Sexual and Reproductive Health and Rights by surfacing new and nontraditional voices, building unconventional partnerships, and creating a new narrative for
FP/RH linked to life choices and economic aspirations.
• Udaan Collaborative: The intervention is focused on building the largest network of
stakeholders, including obstetricians and gynecologists, general physicians, and pharmacists
in the private sector, to increase young people's use of family planning products and services.
• Yash Entrepreneurs Program: This intervention will identify and support medium—and latestage enterprises that scale innovations in the family planning and reproductive health
domain.
• Enhancing the Value of Girl Child - through social action initiatives in colleges and vocational
institutes: This intervention aims to address regressive gender norms and support the rights
of women and girls. By engaging colleges and vocational institutions through collaboration
and partnership, the intervention aims to create a system of processes that will cause a change
to ensure healthy lives and promote the well-being of all girls.
• Special Cell and Community Approach (SCCA): The intervention combats GBV by focusing on
prevention, strengthening state response, and increasing SRH knowledge among young GBV
survivors in the state of Assam.
• Protecting rights of girls in India in the COVID era by preventing CEFM (Child, Early, and Forced
Marriage): The intervention promotes delaying the age of marriage of adolescent girls by
creating an enabling and empowering ecosystem, and preventing early forced marriage, by
connecting them to social protection schemes including livelihood opportunities, conducting
trainings on life skills development and sensitizing with faith-based leaders on harm caused
by CEFM.
8
• Technical Assistance to Govt. of India - Ayushman Bharat School Health and Wellness
Programme: A flagship program by the Govt. of India to increase awareness of health issues,
including sexual and reproductive health (SRH) among school-going adolescents. MCGL IndiaYash is currently providing technical assistance to the government in 10 states to ensure the
smooth implementation of the program within government schools.
• Contraceptive Security: Elevating the Govt. Of India's supply chain, our initiative integrates
advanced Family Planning Logistics Management Information Systems (FPLMIS) tools for resilient
last-mile delivery of contraceptives. Amidst climate challenges in flood and drought-prone areas,
efforts uniquely safeguard contraceptive security while concurrently prioritizing eco-sustainability
by implementing measures to reduce carbon emissions in the supply chain.
• Delhi Learning Lab: Delhi Learning Lab integrates products, services, and best practices in
sexual reproductive health and family planning in a community-centered youth-led hub for
positive youth development.
• Expansion of Contraceptive Choices: The intervention aims to facilitate access to an expanded
basket of contraceptive options, by introducing newer contraceptive methods – contraceptive
implants and DMPA Subcutaneous and increasing awareness and knowledge in the community
regarding the need for family planning. MCGL is providing technical support to the states of Gujarat
and Tamil Nadu to ensure the provision of quality and voluntary family planning services, as well
as the strengthening of facilities and districts through clinical and programmatic support. The goal
of this intervention is to ensure the smooth introduction of newer contraceptives and promote
voluntary family planning services in the region along with adherence to guidelines laid out by the
Government of India in alignment with FP 2030.
USAID India, with the MOMENTUM Country and Global Leadership (MCGL) field support mechanism,
is implementing a program focused on Family Planning (FP) and Reproductive Health (RH) for young
populations (YPs) that will contribute to the achievement of national FP goals and is aligned with USAID
India’s Country Development Cooperation Strategies (CDCS) goal of improving the health and
resilience of India’s marginalized populations. The project aims to demonstrate innovative models to
improve young people’s FP/RH outcomes. The project promotes the adaptation of promising models
of high-impact interventions demonstrated in previous programs. This project is expected to catalyze
lessons learned from USAID’s previous investments in FP/RH for youth. The project will leave behind
replicable and scalable models for young people so that they take care of their own sexual
reproductive health needs and well-being, irrespective of their socio-demographic and economic
differences. This program is aligned with USAID’s priority of leading India to achieve progress towards
sustainable development by building local institutional capacities steering USAID India’s strategic
approaches of promoting inclusive development, harnessing private sector engagement, and
leveraging high-level government of India commitments and leadership.
MCGL India-Yash interventions
• Youth-led Movement: The intervention aims to build a multi-state Family Planning and Sexual
and Reproductive Health and Rights youth-led movement through surfacing new and non
traditional voices, building unconventional partnerships, and creating a new narrative for
FP/RH linked to life choices and economic aspirations.
• Udaan Collaborative: The intervention is focused on building the largest network of stakeholders,
including obstetrics and gynecologists, general physicians, and pharmacists in the private
sector, to increase the usage of family planning products and services by young people.
• Yash Entrepreneurs Program: The intervention identifies and supports medium and late-stage
enterprises to scale family planning and reproductive health innovations.
• Enhancing the Value of Girl Child - through social action initiatives in colleges and vocational
institutes: This intervention aims to address regressive gender norms and support the rights of
women and girls. By engaging colleges and vocational institutions through collaboration and
partnership, the intervention aims to create a system of processes that will cause a change to
ensure healthy lives and promote the well-being of all girls.
• Special Cell and Community Approach (SCCA): The intervention is working towards combating
GBV by focusing on prevention, strengthening state response, and increasing SRH knowledge
amongst young GBV survivors in Assam.
• Protecting the rights of girls in India in the COVID era by preventing CEFM (Child, Early, and Forced
Marriage): The intervention promotes delaying the age of marriage of adolescent girls by
creating an enabling and empowering ecosystem and preventing early forced marriage by
connecting them to social protection schemes including livelihood opportunities, conducting
trainings on life skills development, and sensitizing with faith-based leaders on harm caused
by CEFM.
• Technical Assistance to Govt. of India - Ayushman Bharat School Health and Wellness
Programme: A flagship program by the Govt. of India to increase awareness of health issues,
7
including sexual and reproductive health (SRH) among school-going adolescents. MCGL India
Yash is currently providing technical assistance to the government in 10 states to ensure the
smooth implementation of the program within government schools.
• Contraceptive Security: Elevating the Govt. Of India's supply chain, our initiative integrates
advanced Family Planning Logistics Management Information Systems (FPLMIS) tools for
resilient last-mile delivery of contraceptives. Amidst climate challenges in flood and drought
prone areas, efforts uniquely safeguard contraceptive security while concurrently prioritizing
eco-sustainability by implementing measures to reduce carbon emissions in the supply chain.
• Delhi Learning Lab: A comprehensive demonstration site developed to implement learnings
from across all interventions of MCGL India-Yash.
• Expansion of Contraceptive Choices: The intervention aims to facilitate access to an expanded
basket of contraceptive options by introducing newer contraceptive methods – contraceptive
implants and DMPA Subcutaneous and increasing awareness and knowledge in the community
regarding the need for family planning. MCGL India-Yash is providing technical support to the
states of Gujarat and Tamil Nadu to ensure the provision of quality and voluntary family
planning services and strengthen facilities and districts through clinical and programmatic
support. This intervention aimsto ensure the smooth introduction of newer contraceptives and
promote voluntary family planning services in the region, along with adherence to guidelines
laid out by the Government of India in alignment with FP 2030.
USAID India with the MOMENTUM Country and Global Leadership (MCGL) field support mechanism is
implementing a program focused on Family Planning (FP) and Reproductive Health (RH) for young
populations (YPs) that will contribute to the achievement of national FP goals and is aligned with
USAID India’s Country Development Cooperation Strategies (CDCS) goal of improving the health and
resilience of India’s marginalized populations. The project’s vision is to demonstrate innovative
models to improve young people’s FP/RH outcomes. The project promotes the adaptation of
promising models of high impact interventions demonstrated in previous programs. This project is
expected to catalyze lessons learned from USAID’s previous investments in FP/RH for youth. The
project will leave behind replicable and scalable models for young people so that they take care of
their own sexual reproductive health needs and wellbeing, irrespective of their socio-demographic
and economic differences. This program is aligned with USAID’s priority of leading India to achieve
progress towards sustainable development by building local institutional capacities, and steering
USAID India’s strategic approaches of promoting inclusive development, harnessing private sector
engagement, and leveraging high-level government of India commitments and leadership.
MCGL India-Yash interventions
• Youth-led Movement: The intervention aims to build a multi-state Family Planning and Sexual
and Reproductive Health and Rights youth-led movement, through surfacing new and nontraditional voices, building unconventional partnerships, and creating a new narrative for
FP/RH linked to life choices and economic aspirations.
• Udaan Collaborative: The intervention is focused on building the largest network of
stakeholders including obstetrics and gynecologists, general physicians and pharmacists in the
private sector, to increase usage of family planning products and services by young people.
• Yash Entrepreneurs Program: The intervention will identify and support medium and latestage enterprises to scale innovations in the family planning and reproductive health domain.
• Enhancing the Value of Girl Child - through social action initiatives in colleges and vocational
institutes: This intervention aims to address regressive gender norms and support the rights of
women and girls. By engaging colleges and vocational institutions, through collaboration and
partnership, the intervention aims to create a system of processes that will cause a change to
ensure healthy lives and promote well-being of all girls.
• Special Cell and Community Approach (SCCA): The intervention is working towards combating
GBV by focusing on prevention and strengthening state response along with increasing SRH
knowledge amongst young GBV survivors in the state of Assam.
• Protecting rights of girls in India in the COVID era by preventing CEFM (Child, Early and Forced
Marriage): The intervention promotes delaying the age of marriage of adolescent girls by
creating an enabling and empowering ecosystem, and preventing early forced marriage, by
connecting them to social protection schemes including livelihood opportunities, conducting
trainings on life skills development and sensitizing with faith-based leaders on harm caused by
CEFM.
• Technical Assistance to Govt. of India - Ayushman Bharat School Health and Wellness
Programme: A flagship program by the Govt. of India to increase awareness on health issues,
including sexual and reproductive health (SRH) among school-going adolescents. MCGL IndiaYash is currently providing technical assistance to the government in 10 states to ensure the
smooth implementation of the program within government schools.
• Contraceptive Security: Elevating the Govt. Of India's supply chain, our initiative integrates
advanced Family Planning Logistics Management Information Systems (FPLMIS) tools for resilient
last-mile delivery of contraceptives. Amidst climate challenges in flood and drought-prone areas,
efforts uniquely safeguard contraceptive security while concurrently prioritizing eco-sustainability
by implementing measures to reduce carbon emissions in the supply chain.
8
• Delhi Learning Lab: A comprehensive demonstration site integrating products, services, and
best practices in sexual reproductive health and family planning in a community centered
youth led hub for positive youth development.
• Expansion of Contraceptive Choices: The intervention aims to facilitate access to an expanded
basket of contraceptive options, by introducing newer contraceptive methods – contraceptive
implants and DMPA subcutaneous injections and increasing awareness and knowledge in the
community regarding the need for family planning. MCGL is providing technical support to the
states of Gujarat and Tamil Nadu to ensure the provision of quality and voluntary family planning
services, as well as the strengthening of facilities and districts through clinical and programmatic
support. The goal of this intervention is to ensure the smooth introduction of newer contraceptives
and promote voluntary family planning services in the region along with adherence to guidelines
laid out by the Government of India in alignment with FP 2030.

"""

# Extract CARS components
cars_components = extract_cars_components(test_text, model, tokenizer, label_map)

# Print the extracted components
for component, sentences in cars_components.items():
    print(f"\n{component}:")
    for s in sentences:
        print(f"- {s}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.645110,0.250000,0.100000,0.062500,0.250000
2,1.634300,1.710128,0.375000,0.333333,0.312500,0.375000
3,1.542200,1.502545,0.250000,0.111111,0.071429,0.250000
4,1.360600,1.262200,0.500000,0.648810,1.000000,0.500000
5,1.193900,1.132097,0.625000,0.666667,0.750000,0.625000



Evaluation results:
{'eval_loss': 1.1320973634719849, 'eval_accuracy': 0.625, 'eval_f1': 0.6666666666666667, 'eval_precision': 0.75, 'eval_recall': 0.625, 'eval_runtime': 1.0689, 'eval_samples_per_second': 7.484, 'eval_steps_per_second': 0.936, 'epoch': 5.0}


Cause:
- 
Guidance: This section should briefly state the results the activity seeks to achieve and the relationship 
of these results to the USAID/India’s goal of Ending Preventable Maternal and Child Death (EPCMD), 
FP2030, TB-free India and AIDS-free generation in India.
- The project’s vision is to demonstrate innovative 
models to improve young people’s FP/RH outcomes.
- The 
project will leave behind replicable and scalable models for young people so that they take care of 
their own sexual reproductive health needs and well-being, irrespective of their socio-demographic 
and economic differences.
- • Yash Entrepreneurs Program: This intervention will identify and support medium—and latestage enterprises that scale innova

test code 6 pm 19th Sept

In [3]:
import pandas as pd
import numpy as np
import torch
import nltk
nltk.download('punkt')

from nltk.tokenize import sent_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import (
    BertTokenizerFast,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments,
)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# =====================================
# Step 1: Prepare the Dataset
# =====================================

# Your CARS dataset in JSON format
cars_dataset = [
  {
    "event": "USAID partners with India to build equitable and inclusive healthcare access and climate-resilient health services.",
    "cause": "Recognizing the need for a robust healthcare system serving India's diverse population and aligning with India's National Health Policy.",
    "action": "Fostering partnerships, leveraging technology and innovation, and focusing on equitable healthcare access.",
    "result": "Improved health outcomes, strengthened health systems, and equitable access to quality healthcare.",
    "significance": "Demonstrates scalable global development solutions and enhances global health security."
  },
  {
    "event": "Creation of the world's first health-focused blended financing facility by USAID and local partners.",
    "cause": "Need for innovative financing to support healthcare enterprises and scale healthcare innovations.",
    "action": "Establishing a financing facility offering grants and debt financing to social entrepreneurs.",
    "result": "Invested in over 90 innovations, benefiting more than 43 million people and leveraging over $270 million in investments.",
    "significance": "Accelerated health innovation scaling and established a replicable financing model globally."
  },
  {
    "event": "Development and nationwide scaling of an AI tool to scan media for disease emergence.",
    "cause": "Need for enhanced disease surveillance and early outbreak detection.",
    "action": "Developing an AI tool to scan media for signs of disease and integrating it into national platforms.",
    "result": "Scanned 65 million articles in 2023, helping prevent an estimated 230 outbreaks.",
    "significance": "Improved global health security and provided a model for other countries."
  },
  {
    "event": "Scaling of maternal and child health models in India and Eastern Africa.",
    "cause": "High rates of infant respiratory distress and maternal deaths due to postpartum hemorrhage.",
    "action": "Demonstrated use of CPAP machines for infants and introduced new drugs and devices to reduce maternal deaths.",
    "result": "Improved maternal and child health outcomes and scaled health models in multiple regions.",
    "significance": "Contributed to reducing mortality rates and provided scalable health solutions globally."
  },
  {
    "event": "Transition of USAID HIV programs to models of entrepreneurship and empowerment for the LGBTQI+ community.",
    "cause": "Need to empower the LGBTQI+ community beyond traditional healthcare models and reduce stigma.",
    "action": "Established comprehensive clinics, supported civil society organizations, and promoted LGBTQI+ social enterprises.",
    "result": "Empowered the community with safe healthcare access and opportunities for self-sufficiency.",
    "significance": "Advanced equality, inclusion, and influenced global approaches to LGBTQI+ healthcare."
  },
  {
    "event": "Partnership with the Government of India to develop climate-resilient healthcare services.",
    "cause": "Impact of climate change on health and the necessity for sustainable healthcare infrastructure.",
    "action": "Developed solar-powered facilities, smart water usage, waste management, and greener supply chains; created state action plans.",
    "result": "Enhanced sustainability and resilience of healthcare services across 13 states.",
    "significance": "Set a precedent for integrating climate resilience into healthcare, aiding global efforts against climate change."
  },
  {
    "event": "Development of online platforms and networks to support young people's health decision-making.",
    "cause": "Need to improve access to healthcare information and empower youth in health decisions.",
    "action": "Helped develop digital platforms providing health information and products to youth.",
    "result": "Enhanced access to health resources and improved decision-making among young people.",
    "significance": "Empowered youth, contributing to long-term positive health outcomes and demonstrating effective use of technology."
  },
  {
    "event": "Facilitation of health learning exchanges between India and other countries.",
    "cause": "Need to share best practices and collaborate on global health challenges.",
    "action": "Organized exchanges with governments of Bangladesh, Fiji, Indonesia, Nepal, Nigeria, the Philippines, and Sri Lanka.",
    "result": "Enhanced collaboration, adoption of successful health practices, and strengthened global health networks.",
    "significance": "Promoted international cooperation, contributing to improved health outcomes globally."
  },
  {
    "event": "Embedding inclusive development principles in USAID's work at every level.",
    "cause": "Importance of diversity, equity, and accessibility in development programs.",
    "action": "Worked with community-based organizations and marginalized groups; implemented inclusive monitoring and evaluation processes.",
    "result": "Increased demand for equitable healthcare services and improved inclusivity in interventions.",
    "significance": "Strengthened impact of initiatives and set standards for inclusive practices in international development."
  },
  {
    "event": "USAID's efforts to enhance global health security and strengthen local health systems.",
    "cause": "Threat of global health crises and the need for robust local health systems.",
    "action": "Strengthened disease surveillance, responded to emerging infections, mobilized resources, and promoted research.",
    "result": "Improved ability to manage health challenges and prevent potential outbreaks.",
    "significance": "Reduced risk of global health emergencies and contributed to safer, healthier communities worldwide."
  },
  {
    "event": "USAID's collaboration to support TB elimination and HIV targets in alignment with India's health goals.",
    "cause": "The prevalence of TB and HIV in India and the need to meet elimination targets.",
    "action": "Supporting programs focused on TB elimination and HIV prevention and treatment.",
    "result": "Progress toward meeting TB elimination and HIV targets in India.",
    "significance": "Contributed to improved public health and aligned with national and global health priorities."
  },
  {
    "event": "USAID's work to ensure local voices guide the formulation of equitable and effective responses.",
    "cause": "Need for community engagement to enhance the effectiveness of health programs.",
    "action": "Collaborated with local communities, marginalized populations, and tribal groups to drive demand for comprehensive healthcare.",
    "result": "Programs that are more responsive to local needs and increased utilization of health services.",
    "significance": "Enhanced community ownership and sustainability of health interventions."
  },
  {
    "event": "Implementation of monitoring and evaluation processes to assess the impact of inclusive development.",
    "cause": "Need to measure the effectiveness of incorporating inclusive development into healthcare interventions.",
    "action": "Developed and implemented monitoring and evaluation frameworks focusing on inclusivity.",
    "result": "Obtained data on the impact of inclusive practices, informing future program improvements.",
    "significance": "Provided evidence to support the effectiveness of inclusive development, influencing policy and program design."
  },
  {
    "event": "USAID's support for telemedicine and mental health initiatives.",
    "cause": "Challenges in accessing healthcare services, especially in remote areas, and increasing mental health concerns.",
    "action": "Promoted telemedicine solutions and mental health programs through partnerships and technology.",
    "result": "Improved access to healthcare and mental health services for underserved populations.",
    "significance": "Addressed critical gaps in healthcare delivery and contributed to overall well-being."
  },
  {
    "event": "Advancement of innovative healthcare models and high-impact solutions through partnerships.",
    "cause": "Complex health challenges in India requiring novel approaches.",
    "action": "Tested and piloted healthcare models and innovations with public and private sector entities.",
    "result": "Development of scalable solutions to health challenges in India.",
    "significance": "Enabled replication of successful models worldwide, enhancing global health outcomes."
  }
]

# Convert the dataset into a DataFrame for easier manipulation
data_rows = []
for entry in cars_dataset:
    # Split each component into sentences (if necessary)
    for component in ['event', 'cause', 'action', 'result', 'significance']:
        text = entry[component]
        sentences = sent_tokenize(text)
        for sentence in sentences:
            data_rows.append({'sentence': sentence, 'label': component.capitalize()})

df = pd.DataFrame(data_rows)

# Map labels to integers
label_map = {'Event': 0, 'Cause': 1, 'Action': 2, 'Result': 3, 'Significance': 4}
df['label'] = df['label'].map(label_map)

# Shuffle the DataFrame
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Split the data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['sentence'].tolist(),
    df['label'].tolist(),
    test_size=0.1,
    random_state=42
)

# =====================================
# Step 2: Tokenization
# =====================================

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(
    train_texts,
    truncation=True,
    padding=True,
    max_length=128
)

val_encodings = tokenizer(
    val_texts,
    truncation=True,
    padding=True,
    max_length=128
)

# =====================================
# Step 3: Create Dataset Class
# =====================================

class CARSDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # Return CPU tensors
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CARSDataset(train_encodings, train_labels)
val_dataset = CARSDataset(val_encodings, val_labels)

# =====================================
# Step 4: Initialize the Model
# =====================================

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(label_map),
    output_attentions=False,
    output_hidden_states=False
).to(device)

# =====================================
# Step 5: Training Arguments
# =====================================

training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    num_train_epochs=5,              # Total number of training epochs
    per_device_train_batch_size=8,   # Batch size per device during training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    warmup_steps=50,                 # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=10,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
)

# =====================================
# Step 6: Define Metrics
# =====================================

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='weighted', zero_division=0
    )
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# =====================================
# Step 7: Initialize Trainer
# =====================================

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# =====================================
# Step 8: Train the Model
# =====================================

trainer.train()

# Save the model
trainer.save_model('./trained_model')
tokenizer.save_pretrained('./trained_model')

# =====================================
# Step 9: Evaluate the Model
# =====================================

eval_results = trainer.evaluate()
print(f"\nEvaluation results:\n{eval_results}\n")

# =====================================
# Step 10: Inference on New Texts
# =====================================

def extract_cars_units(text, model, tokenizer, label_map):
    # Split the text into sentences
    sentences = sent_tokenize(text)

    # Tokenize sentences
    encodings = tokenizer(
        sentences,
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors='pt'
    ).to(device)  # Move tensors to the appropriate device

    # Make predictions
    with torch.no_grad():
        outputs = model(**encodings)
        logits = outputs.logits
        predicted_class_ids = torch.argmax(logits, dim=1)
        predicted_labels = predicted_class_ids.cpu().numpy()

    # Map IDs to labels
    id_to_label = {v: k for k, v in label_map.items()}
    predicted_labels = [id_to_label[id] for id in predicted_labels]

    # Initialize list to hold CARS units
    cars_units = []
    current_unit = {'Cause': '', 'Action': '', 'Result': '', 'Significance': '', 'Event': ''}
    unit_started = False  # Flag to indicate if a unit has started

    # Iterate over sentences and their predicted labels
    for sentence, label in zip(sentences, predicted_labels):
        if label == 'Cause':
            # If there's already content in current_unit, save it before starting a new one
            if unit_started:
                cars_units.append(current_unit)
                # Reset current_unit
                current_unit = {'Cause': '', 'Action': '', 'Result': '', 'Significance': '', 'Event': ''}
            # Start a new unit with the Cause
            current_unit['Cause'] = sentence
            unit_started = True
        elif label in ['Action', 'Result', 'Significance', 'Event']:
            if not unit_started:
                # Start a new unit even if 'Cause' is missing
                current_unit = {'Cause': '', 'Action': '', 'Result': '', 'Significance': '', 'Event': ''}
                unit_started = True
            # Add the sentence to the appropriate component
            current_unit[label] += (' ' + sentence).strip()
        else:
            # For labels not in CARS (if any), you can choose to handle them or ignore
            pass

    # Append the last unit if it's not empty
    if unit_started:
        cars_units.append(current_unit)

    return cars_units

# Example usage:

# Load the trained model and tokenizer (if not already in memory)
# model = BertForSequenceClassification.from_pretrained('./trained_model').to(device)
# tokenizer = BertTokenizerFast.from_pretrained('./trained_model')

# Sample test text
test_text = """
Guidance: This section should briefly state the results the activity seeks to achieve and the relationship
of these results to the USAID/India’s goal of Ending Preventable Maternal and Child Death (EPCMD),
FP2030, TB-free India and AIDS-free generation in India. Provide clear and precise descriptions of the
guiding principle for this QPR, e.g. highlights of activities, key achievements of the reporting period,
major plans of the next quarter, etc. Restrict to TWO pages only.
USAID India with the MOMENTUM Country and Global Leadership (MCGL) field support mechanism is
implementing a program focused on Family Planning (FP) and Reproductive Health (RH) for young
populations (YPs) that will contribute to the achievement of national FP goals and is aligned with
USAID India’s Country Development Cooperation Strategies (CDCS) goal of improving the health and
resilience of India’s marginalized populations. The project’s vision is to demonstrate innovative
models to improve young people’s FP/RH outcomes. The project promotes the adaptation of
promising models of high-impact interventions demonstrated in previous programs. This project is
expected to catalyze lessons learned from USAID’s previous investments in FP/RH for youth. The
project will leave behind replicable and scalable models for young people so that they take care of
their own sexual reproductive health needs and well-being, irrespective of their socio-demographic
and economic differences. This program is aligned with USAID’s priority of leading India to achieve
progress towards sustainable development by building local institutional capacities, and steering
USAID India’s strategic approaches of promoting inclusive development, harnessing private sector
engagement, and leveraging high-level government of India commitments and leadership.
MCGL India-Yash interventions:
• Youth-led Movement: The intervention aims to build a multi-state youth-led movement for
Family Planning and Sexual and Reproductive Health and Rights by surfacing new and non traditional voices, building unconventional partnerships, and creating a new narrative for
FP/RH linked to life choices and economic aspirations.
• Udaan Collaborative: The intervention is focused on building the largest network of
stakeholders, including obstetricians and gynecologists, general physicians, and pharmacists
in the private sector, to increase young people's use of family planning products and services.
• Yash Entrepreneurs Program: This intervention will identify and support medium—and late stage enterprises that scale innovations in the family planning and reproductive health
domain.
• Enhancing the Value of Girl Child - through social action initiatives in colleges and vocational
institutes: This intervention aims to address regressive gender norms and support the rights
of women and girls. By engaging colleges and vocational institutions through collaboration
and partnership, the intervention aims to create a system of processes that will cause a change
to ensure healthy lives and promote the well-being of all girls.
• Special Cell and Community Approach (SCCA): The intervention combats GBV by focusing on
prevention, strengthening state response, and increasing SRH knowledge among young GBV
survivors in the state of Assam.
• Protecting rights of girls in India in the COVID era by preventing CEFM (Child, Early, and Forced
Marriage): The intervention promotes delaying the age of marriage of adolescent girls by
creating an enabling and empowering ecosystem, and preventing early forced marriage, by
connecting them to social protection schemes including livelihood opportunities, conducting
trainings on life skills development and sensitizing with faith-based leaders on harm caused
by CEFM.
8
• Technical Assistance to Govt. of India - Ayushman Bharat School Health and Wellness
Programme: A flagship program by the Govt. of India to increase awareness of health issues,
including sexual and reproductive health (SRH) among school-going adolescents. MCGL India Yash is currently providing technical assistance to the government in 10 states to ensure the
smooth implementation of the program within government schools.
• Contraceptive Security: Elevating the Govt. Of India's supply chain, our initiative integrates
advanced Family Planning Logistics Management Information Systems (FPLMIS) tools for resilient
last-mile delivery of contraceptives. Amidst climate challenges in flood and drought-prone areas,
efforts uniquely safeguard contraceptive security while concurrently prioritizing eco-sustainability
by implementing measures to reduce carbon emissions in the supply chain.
• Delhi Learning Lab: Delhi Learning Lab integrates products, services, and best practices in
sexual reproductive health and family planning in a community-centered youth-led hub for
positive youth development.
• Expansion of Contraceptive Choices: The intervention aims to facilitate access to an expanded
basket of contraceptive options, by introducing newer contraceptive methods – contraceptive
implants and DMPA Subcutaneous and increasing awareness and knowledge in the community
regarding the need for family planning. MCGL is providing technical support to the states of Gujarat
and Tamil Nadu to ensure the provision of quality and voluntary family planning services, as well
as the strengthening of facilities and districts through clinical and programmatic support. The goal
of this intervention is to ensure the smooth introduction of newer contraceptives and promote
voluntary family planning services in the region along with adherence to guidelines laid out by the
Government of India in alignment with FP 2030.
USAID India, with the MOMENTUM Country and Global Leadership (MCGL) field support mechanism,
is implementing a program focused on Family Planning (FP) and Reproductive Health (RH) for young
populations (YPs) that will contribute to the achievement of national FP goals and is aligned with USAID
India’s Country Development Cooperation Strategies (CDCS) goal of improving the health and
resilience of India’s marginalized populations. The project aims to demonstrate innovative models to
improve young people’s FP/RH outcomes. The project promotes the adaptation of promising models
of high-impact interventions demonstrated in previous programs. This project is expected to catalyze
lessons learned from USAID’s previous investments in FP/RH for youth. The project will leave behind
replicable and scalable models for young people so that they take care of their own sexual
reproductive health needs and well-being, irrespective of their socio-demographic and economic
differences. This program is aligned with USAID’s priority of leading India to achieve progress towards
sustainable development by building local institutional capacities steering USAID India’s strategic
approaches of promoting inclusive development, harnessing private sector engagement, and
leveraging high-level government of India commitments and leadership.
MCGL India-Yash interventions
• Youth-led Movement: The intervention aims to build a multi-state Family Planning and Sexual
and Reproductive Health and Rights youth-led movement through surfacing new and non
traditional voices, building unconventional partnerships, and creating a new narrative for
FP/RH linked to life choices and economic aspirations.
• Udaan Collaborative: The intervention is focused on building the largest network of stakeholders,
including obstetrics and gynecologists, general physicians, and pharmacists in the private
sector, to increase the usage of family planning products and services by young people.
• Yash Entrepreneurs Program: The intervention identifies and supports medium and late-stage
enterprises to scale family planning and reproductive health innovations.
• Enhancing the Value of Girl Child - through social action initiatives in colleges and vocational
institutes: This intervention aims to address regressive gender norms and support the rights of
women and girls. By engaging colleges and vocational institutions through collaboration and
partnership, the intervention aims to create a system of processes that will cause a change to
ensure healthy lives and promote the well-being of all girls.
• Special Cell and Community Approach (SCCA): The intervention is working towards combating
GBV by focusing on prevention, strengthening state response, and increasing SRH knowledge
amongst young GBV survivors in Assam.
• Protecting the rights of girls in India in the COVID era by preventing CEFM (Child, Early, and Forced
Marriage): The intervention promotes delaying the age of marriage of adolescent girls by
creating an enabling and empowering ecosystem and preventing early forced marriage by
connecting them to social protection schemes including livelihood opportunities, conducting
trainings on life skills development, and sensitizing with faith-based leaders on harm caused
by CEFM.
• Technical Assistance to Govt. of India - Ayushman Bharat School Health and Wellness
Programme: A flagship program by the Govt. of India to increase awareness of health issues,
7
including sexual and reproductive health (SRH) among school-going adolescents. MCGL India
Yash is currently providing technical assistance to the government in 10 states to ensure the
smooth implementation of the program within government schools.
• Contraceptive Security: Elevating the Govt. Of India's supply chain, our initiative integrates
advanced Family Planning Logistics Management Information Systems (FPLMIS) tools for
resilient last-mile delivery of contraceptives. Amidst climate challenges in flood and drought
prone areas, efforts uniquely safeguard contraceptive security while concurrently prioritizing
eco-sustainability by implementing measures to reduce carbon emissions in the supply chain.
• Delhi Learning Lab: A comprehensive demonstration site developed to implement learnings
from across all interventions of MCGL India-Yash.
• Expansion of Contraceptive Choices: The intervention aims to facilitate access to an expanded
basket of contraceptive options by introducing newer contraceptive methods – contraceptive
implants and DMPA Subcutaneous and increasing awareness and knowledge in the community
regarding the need for family planning. MCGL India-Yash is providing technical support to the
states of Gujarat and Tamil Nadu to ensure the provision of quality and voluntary family
planning services and strengthen facilities and districts through clinical and programmatic
support. This intervention aimsto ensure the smooth introduction of newer contraceptives and
promote voluntary family planning services in the region, along with adherence to guidelines
laid out by the Government of India in alignment with FP 2030.
USAID India with the MOMENTUM Country and Global Leadership (MCGL) field support mechanism is
implementing a program focused on Family Planning (FP) and Reproductive Health (RH) for young
populations (YPs) that will contribute to the achievement of national FP goals and is aligned with
USAID India’s Country Development Cooperation Strategies (CDCS) goal of improving the health and
resilience of India’s marginalized populations. The project’s vision is to demonstrate innovative
models to improve young people’s FP/RH outcomes. The project promotes the adaptation of
promising models of high impact interventions demonstrated in previous programs. This project is
expected to catalyze lessons learned from USAID’s previous investments in FP/RH for youth. The
project will leave behind replicable and scalable models for young people so that they take care of
their own sexual reproductive health needs and wellbeing, irrespective of their socio-demographic
and economic differences. This program is aligned with USAID’s priority of leading India to achieve
progress towards sustainable development by building local institutional capacities, and steering
USAID India’s strategic approaches of promoting inclusive development, harnessing private sector
engagement, and leveraging high-level government of India commitments and leadership.
MCGL India-Yash interventions
• Youth-led Movement: The intervention aims to build a multi-state Family Planning and Sexual
and Reproductive Health and Rights youth-led movement, through surfacing new and non traditional voices, building unconventional partnerships, and creating a new narrative for
FP/RH linked to life choices and economic aspirations.
• Udaan Collaborative: The intervention is focused on building the largest network of
stakeholders including obstetrics and gynecologists, general physicians and pharmacists in the
private sector, to increase usage of family planning products and services by young people.
• Yash Entrepreneurs Program: The intervention will identify and support medium and late stage enterprises to scale innovations in the family planning and reproductive health domain.
• Enhancing the Value of Girl Child - through social action initiatives in colleges and vocational
institutes: This intervention aims to address regressive gender norms and support the rights of
women and girls. By engaging colleges and vocational institutions, through collaboration and
partnership, the intervention aims to create a system of processes that will cause a change to
ensure healthy lives and promote well-being of all girls.
• Special Cell and Community Approach (SCCA): The intervention is working towards combating
GBV by focusing on prevention and strengthening state response along with increasing SRH
knowledge amongst young GBV survivors in the state of Assam.
• Protecting rights of girls in India in the COVID era by preventing CEFM (Child, Early and Forced
Marriage): The intervention promotes delaying the age of marriage of adolescent girls by
creating an enabling and empowering ecosystem, and preventing early forced marriage, by
connecting them to social protection schemes including livelihood opportunities, conducting
trainings on life skills development and sensitizing with faith-based leaders on harm caused by
CEFM.
• Technical Assistance to Govt. of India - Ayushman Bharat School Health and Wellness
Programme: A flagship program by the Govt. of India to increase awareness on health issues,
including sexual and reproductive health (SRH) among school-going adolescents. MCGL India Yash is currently providing technical assistance to the government in 10 states to ensure the
smooth implementation of the program within government schools.
• Contraceptive Security: Elevating the Govt. Of India's supply chain, our initiative integrates
advanced Family Planning Logistics Management Information Systems (FPLMIS) tools for resilient
last-mile delivery of contraceptives. Amidst climate challenges in flood and drought-prone areas,
efforts uniquely safeguard contraceptive security while concurrently prioritizing eco-sustainability
by implementing measures to reduce carbon emissions in the supply chain.
8
• Delhi Learning Lab: A comprehensive demonstration site integrating products, services, and
best practices in sexual reproductive health and family planning in a community centered
youth led hub for positive youth development.
• Expansion of Contraceptive Choices: The intervention aims to facilitate access to an expanded
basket of contraceptive options, by introducing newer contraceptive methods – contraceptive
implants and DMPA subcutaneous injections and increasing awareness and knowledge in the
community regarding the need for family planning. MCGL is providing technical support to the
states of Gujarat and Tamil Nadu to ensure the provision of quality and voluntary family planning
services, as well as the strengthening of facilities and districts through clinical and programmatic
support. The goal of this intervention is to ensure the smooth introduction of newer contraceptives
and promote voluntary family planning services in the region along with adherence to guidelines
laid out by the Government of India in alignment with FP 2030.
"""

# Extract CARS units
cars_units = extract_cars_units(test_text, model, tokenizer, label_map)

# Print the extracted units
for idx, unit in enumerate(cars_units, 1):
    print(f"\nCARS Unit {idx}:")
    for component in ['Cause', 'Action', 'Result', 'Significance', 'Event']:
        if unit[component]:
            print(f"{component}: {unit[component]}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` inst

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.489471,0.625000,0.480769,0.390625,0.625000
2,1.639800,1.552883,0.250000,0.312500,0.416667,0.250000
3,1.560700,1.461946,0.250000,0.125000,0.083333,0.250000
4,1.343900,1.225670,0.750000,0.843750,1.000000,0.750000
5,1.125100,1.214112,0.625000,0.682143,0.916667,0.625000



Evaluation results:
{'eval_loss': 1.2256698608398438, 'eval_accuracy': 0.75, 'eval_f1': 0.84375, 'eval_precision': 1.0, 'eval_recall': 0.75, 'eval_runtime': 0.6592, 'eval_samples_per_second': 12.136, 'eval_steps_per_second': 1.517, 'epoch': 5.0}


CARS Unit 1:
Cause: 
Guidance: This section should briefly state the results the activity seeks to achieve and the relationship 
of these results to the USAID/India’s goal of Ending Preventable Maternal and Child Death (EPCMD), 
FP2030, TB-free India and AIDS-free generation in India.
Action: Provide clear and precise descriptions of the 
guiding principle for this QPR, e.g.
Result: Restrict to TWO pages only.
Event: highlights of activities, key achievements of the reporting period, 
major plans of the next quarter, etc.USAID India with the MOMENTUM Country and Global Leadership (MCGL) field support mechanism is 
implementing a program focused on Family Planning (FP) and Reproductive Health (RH) for young 
populations (YPs) that will contri